# Calculations of the Effect Size (ES) for each microarray study 

###  Using Hedges' g value, an adjusted Cohen's d  value

$$  {Enrichment} = \bar{X_2}-\bar{X_1}$$

Let Group 1 be 6hSleeping whole brain DBA/2J (D2) strain Expression values and Group 2 be 6hSD whole brain DBA/2J (D2) strain Expression values 

(S mean - SD mean) **(Logged values, so minus gives ratio)** 

$$  {Pooled\ Standard\  Deviation} = \sqrt\frac{(n_1-1)S_1^2 +(n_2-1)S_2^2}{(n_2 +n_2) -2}  $$  

$$  {Cohen's\ d\ value} = \frac{Enrichment}{Pooled\ Standard\ Deviation} $$

$$  {Correction\ Factor (J\ Factor)} = 1- \frac{3}{4df-1} $$

$$  {Hedges'\ g\ value} = Cohen's\ d\ \text{x}\ J\ $$

$$  {Variance\ in\ d (V_d)} = \frac{n_1- +n_2}{n_1 n_2} + \frac{d^2}{2(n_1 +n_2)}  $$

$$  {Variance\ in\ g (V_g)} = J^2\  \text{x}\ V_d  $$

$$  {Standard\ Error\ in\ g (SE_g)} = \sqrt{V_g}  $$

## Setup working environment and import data

In [1]:
import pandas as pd # Dataframes and file IO
import numpy as np # numerical calculations
%cd /Users/Ella1/Desktop/data sets 430AV2


/Users/Ella1/Desktop/data sets 430AV2


In [2]:
prefix = '430AV2_WB_D2_'   # define a prefix to add to column names (mD2ing indexing easier later)

In [3]:
# import the data file to a data frame 'df'
df=pd.read_table('DATASET-GSE9441.txt', delimiter='\t',  index_col=0) #,nrows=500)  
df.shape

(45101, 51)

In [4]:
# remove probes that are know to cross-hybridise to more than one target
df =df[~df.index.str.contains('_x_|_s_')]    #   important reverse selector ~ 
df.shape

(40569, 51)

## Look at column names and then setup filters for grouping columns into S and SD groups

In [5]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239832_AK_S.CEL', 'GSM239833_AK_S.CEL',
       'GSM239834_AK_S.CEL', 'avg-AK_S', 'log_fold-AK_S_vs_AK_SD',
       'fold-AK_S_vs_AK_SD', 'rawp-AK_S_vs_AK_SD', 'adjp-AK_S_vs_AK_SD',
       'GSM239835_AK_SD.CEL', 'GSM239836_AK_SD.CEL', 'GSM239837_AK_SD.CEL',
       'avg-AK_SD', 'GSM239838_B6_S.CEL', 'GSM239839_B6_S.CEL',
       'GSM239840_B6_S.CEL', 'avg-B6_S', 'log_fold-B6_S_vs_B6_SD',
       'fold-B6_S_vs_B6_SD', 'rawp-B6_S_vs_B6_SD', 'adjp-B6_S_vs_B6_SD',
       'GSM239841_B6_SD.CEL', 'GSM239842_B6_SD.CEL', 'GSM239843_B6_SD.CEL',
       'avg-B6_SD', 'GSM239844_D2_S.CEL', 'GSM239845_D2_S.CEL',
       'GSM239846_D2_S.CEL', 'avg-D2_S', 'log_fold-D2_S_vs_D2_SD',
       'fold-D2_S_vs_D2_SD', 'rawp-D2_S_vs_D2_SD', 'adjp-D2_S_vs_D2_SD',
       'G

In [6]:
# define regular expressions for sleep (S) and sleep dep (SD) filters 
s_filt ='D2_S.CEL'
sd_filt ='D2_SD.CEL'

In [7]:
df_s=df.filter(regex= s_filt)
df_s.head()

,GSM239844_D2_S.CEL,GSM239845_D2_S.CEL,GSM239846_D2_S.CEL
Probesets,,,
1427138_at,4.97679,4.76221,4.94363
1425600_a_at,6.74540,6.61719,6.01381
1457168_at,4.31935,4.37179,4.22728
1450135_at,4.19276,4.28367,4.28681
1424014_at,7.75921,7.66804,7.81685


In [8]:
df_sd=df.filter(regex= sd_filt)
df_sd.head()

,GSM239847_D2_SD.CEL,GSM239848_D2_SD.CEL,GSM239849_D2_SD.CEL
Probesets,,,
1427138_at,4.88983,4.62555,4.84447
1425600_a_at,6.59855,6.27891,6.64337
1457168_at,4.33671,4.28326,4.34071
1450135_at,4.05626,3.98732,4.27150
1424014_at,7.71932,7.36670,7.93176


## Calculations 

In [9]:
# Enrichment

df[prefix+'Enrich'] = df.filter(regex=sd_filt).mean(axis=1) - df.filter(regex=s_filt).mean(axis=1)

In [10]:
df[prefix+'Enrich'].head()

Probesets
1427138_at     -0.107593
1425600_a_at    0.048143
1457168_at      0.014087
1450135_at     -0.149387
1424014_at     -0.075440
Name: 430AV2_WB_D2_Enrich, dtype: float64

In [11]:
# Calculating Pooled StDev
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

StdevS = (Scount-1) * df.filter(regex=s_filt).var(axis=1)
StdevSD = (SDcount-1) * df.filter(regex=sd_filt).var(axis=1)

df[prefix+'poolStDev'] = np.sqrt((StdevS+StdevSD)/(Scount+ SDcount-2))

In [12]:
# Calculating Cohen's d
df[prefix+'Cohens_d'] = df[prefix+'Enrich'] / df[prefix+'poolStDev']

In [13]:
#df[prefix+'poolStDev'].head()
df[prefix+'Cohens_d'] .head()

Probesets
1427138_at     -0.833655
1425600_a_at    0.155332
1457168_at      0.249398
1450135_at     -1.340815
1424014_at     -0.361519
Name: 430AV2_WB_D2_Cohens_d, dtype: float64

In [14]:
# Calculating J value (Correction factor)

df[prefix+'J'] = 1-(3/(4*(Scount+SDcount-1)))                              


In [15]:
# Calculating Hedge's g

df[prefix+'Hedges_g'] = df[prefix+'Cohens_d'] * df[prefix+'J']

In [16]:
#df[prefix+'J'].head()
df[prefix+'Hedges_g'] .head()

Probesets
1427138_at     -0.708606
1425600_a_at    0.132032
1457168_at      0.211989
1450135_at     -1.139693
1424014_at     -0.307291
Name: 430AV2_WB_D2_Hedges_g, dtype: float64

In [17]:
# Calculating Var_d
Scount = df.filter(regex=s_filt).count(axis=1)
SDcount = df.filter(regex=sd_filt).count(axis=1)

Ftop1 = Scount + SDcount
Ftop2 = Scount * SDcount
Fbottom1 = np.square(df[prefix+'Cohens_d']) 
Fbottom2 =  2*(Scount + SDcount)


df[prefix+'Var_d'] = (Ftop1/Ftop2) + (Fbottom1 /Fbottom2)

In [18]:
#check output
df[prefix+'Var_d'].head()

Probesets
1427138_at      0.724582
1425600_a_at    0.668677
1457168_at      0.671850
1450135_at      0.816482
1424014_at      0.677558
Name: 430AV2_WB_D2_Var_d, dtype: float64

In [19]:
df[prefix+'Var_g'] = df[prefix+'Var_d'] * np.square(df[prefix+'J'])

In [20]:
# Calculating SEg
df[prefix+'SEg'] = np.sqrt(df[prefix+'Var_g'])

In [21]:
df.sort_values(by= '430AV2_WB_D2_Hedges_g', ascending=False, inplace=True)
df

,Symbol,Definition,Ensembl_id,Entrez_id,Unigene_id,GO-Process,GO-Function,GO-Component,Pathway_info,Putative microRNA binding sites,...,ANOVA-adjp,largest fold,430AV2_WB_D2_Enrich,430AV2_WB_D2_poolStDev,430AV2_WB_D2_Cohens_d,430AV2_WB_D2_J,430AV2_WB_D2_Hedges_g,430AV2_WB_D2_Var_d,430AV2_WB_D2_Var_g,430AV2_WB_D2_SEg
Probesets,,,,,,,,,,,,,,,,,,,,,
1457687_at,Bcl2,B cell leukemia/lymphoma 2 [Source:MGI Symbol;...,ENSMUSG00000057329,12043,NaN,endoplasmic reticulum calcium ion homeostasis ...,transcription factor binding // protein homodi...,endoplasmic reticulum membrane // cytosol // c...,Estrogen signalling:WP1244(WikiPathways) // Se...,"mmu-miR-1(miRanda), mmu-miR-103(pictar), mmu-m...",...,2.496514e-03,0.668400,0.491583,0.022847,21.516050,0.85,18.288642,39.245032,28.354536,5.324898
1423648_at,Pdia6,protein disulfide isomerase associated 6,ENSMUSG00000020571,71853,NaN,cell redox homeostasis,isomerase activity // protein disulfide isomer...,endoplasmic reticulum lumen // melanosome // e...,NaN,"mmu-let-7a(RNAhybrid|miRanda), mmu-let-7b(RNAh...",...,1.492835e-06,1.071163,0.884913,0.051571,17.159100,0.85,14.585235,25.202892,18.209090,4.267211
1416064_a_at,Hspa5,heat shock protein 5,ENSMUSG00000026864,14828,NaN,response to stress // ER overload response,protein binding // nucleotide binding // ATP b...,endoplasmic reticulum lumen // melanosome // e...,MAPK signaling pathway:WP493(WikiPathways),"mmu-miR-1192(TargetScan|miRanda), mmu-miR-1193...",...,1.191694e-07,1.365547,1.019790,0.075551,13.497970,0.85,11.473275,15.849600,11.451336,3.383982
1450843_a_at,Serpinh1,"serine (or cysteine) peptidase inhibitor, clad...",ENSMUSG00000070436,12406,NaN,collagen biosynthetic process // response to s...,protein binding // serine-type endopeptidase i...,cytoplasm // endoplasmic reticulum lumen // en...,Endochondral Ossification:WP1270(WikiPathways),"mmu-miR-1(miRanda), mmu-miR-1192(miRanda), mmu...",...,1.083896e-03,1.040507,0.644510,0.048408,13.314109,0.85,11.316993,15.438792,11.154527,3.339839
1435329_at,Fbxl11,lysine (K)-specific demethylase 2A [Source:MGI...,ENSMUSG00000054611,225876,NaN,transcription // ubiquitin-dependent protein c...,"oxidoreductase activity, acting on single dono...",nucleus,NaN,"mmu-let-7a(miRanda), mmu-let-7b(RNAhybrid|miRa...",...,2.212354e-03,0.549807,0.292340,0.026195,11.160040,0.85,9.486034,11.045541,7.980404,2.824961
1441519_at,Nid1,nidogen 1,NaN,18073,Mm.4691,bioluminescence // positive regulation of cell...,calcium ion binding // protein binding // extr...,extracellular region // proteinaceous extracel...,NaN,NaN,...,5.741901e-01,0.129723,0.091700,0.008463,10.835096,0.85,9.209832,10.449943,7.550084,2.747742
1434595_at,Trim9,tripartite motif-containing 9 [Source:MGI Symb...,ENSMUSG00000021071,94090,NaN,synaptic vesicle exocytosis,protein binding // metal ion binding // zinc i...,cytoplasm // intracellular // synaptosome,NaN,"mmu-miR-10a(RNAhybrid|miRanda), mmu-miR-10b(RN...",...,1.506976e-04,1.212050,0.858760,0.086042,9.980751,0.85,8.483638,8.967949,6.479343,2.545455
1423074_at,Lman2,"lectin, mannose-binding 2",ENSMUSG00000021484,66890,NaN,transport // protein transport,calcium ion binding // sugar binding,Golgi membrane // Golgi apparatus // integral ...,NaN,"mmu-miR-101a(miRanda), mmu-miR-101b(miRanda), ...",...,3.033258e-02,0.410860,0.349423,0.036405,9.598135,0.85,8.158415,8.343683,6.028311,2.455262
1422943_a_at,Hspb1,heat shock protein 1,ENSMUSG00000004951|ENSMUSG00000078915,15507,NaN,response to stress // response to heat,NaN,intracellular // soluble fraction // contracti...,IL-3 Signaling Pathway:WP373(WikiPathways) // ...,"mmu-let-7a(RNAhybrid|miRanda), mmu-let-7b(RNAh...",...,3.040785e-04,1.150387,1.044540,0.109368,9.550722,0.85,8.118114,8.268025,5.973648,2.444105


In [22]:
df.columns

Index(['Symbol', 'Definition', 'Ensembl_id', 'Entrez_id', 'Unigene_id',
       'GO-Process', 'GO-Function', 'GO-Component', 'Pathway_info',
       'Putative microRNA binding sites', 'Select Cellular Compartments',
       'Select Protein Classes', 'GSM239832_AK_S.CEL', 'GSM239833_AK_S.CEL',
       'GSM239834_AK_S.CEL', 'avg-AK_S', 'log_fold-AK_S_vs_AK_SD',
       'fold-AK_S_vs_AK_SD', 'rawp-AK_S_vs_AK_SD', 'adjp-AK_S_vs_AK_SD',
       'GSM239835_AK_SD.CEL', 'GSM239836_AK_SD.CEL', 'GSM239837_AK_SD.CEL',
       'avg-AK_SD', 'GSM239838_B6_S.CEL', 'GSM239839_B6_S.CEL',
       'GSM239840_B6_S.CEL', 'avg-B6_S', 'log_fold-B6_S_vs_B6_SD',
       'fold-B6_S_vs_B6_SD', 'rawp-B6_S_vs_B6_SD', 'adjp-B6_S_vs_B6_SD',
       'GSM239841_B6_SD.CEL', 'GSM239842_B6_SD.CEL', 'GSM239843_B6_SD.CEL',
       'avg-B6_SD', 'GSM239844_D2_S.CEL', 'GSM239845_D2_S.CEL',
       'GSM239846_D2_S.CEL', 'avg-D2_S', 'log_fold-D2_S_vs_D2_SD',
       'fold-D2_S_vs_D2_SD', 'rawp-D2_S_vs_D2_SD', 'adjp-D2_S_vs_D2_SD',
       'G

### Import key file from BioMart and index probesets to MGI gene symbols

In [23]:
dfX=pd.read_table('../FHS project/Sleep notebook Copy/BioMart_Ensmbl_index/mart_export72_430v2430Av2.txt',index_col=[3])
 
dfX.pop('Affy mouse430 2 probeset') # remove 430V2 probeset info (not needed for 430AV2 indexing)
dfX.head(5)

,Ensembl Gene ID,Description,MGI symbol
Affy mouse430a 2 probeset,,,
1417126_a_at,ENSMUSG00000039221,ribosomal protein L22 like 1 [Source:MGI Symbo...,Rpl22l1
NaN,ENSMUSG00000095611,predicted gene 10597 [Source:MGI Symbol;Acc:MG...,Gm10597
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
1417730_at,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1
NaN,ENSMUSG00000061731,exostoses (multiple) 1 [Source:MGI Symbol;Acc:...,Ext1


In [24]:
df_Join = df.join(dfX, how='left', sort=True)
df_FINAL1 = df_Join.groupby('MGI symbol').mean()
df_FINAL1[df_FINAL1.index.duplicated()==True]   # checking that no duplicate entries exist in the dataframe

,GSM239832_AK_S.CEL,GSM239833_AK_S.CEL,GSM239834_AK_S.CEL,avg-AK_S,log_fold-AK_S_vs_AK_SD,fold-AK_S_vs_AK_SD,rawp-AK_S_vs_AK_SD,adjp-AK_S_vs_AK_SD,GSM239835_AK_SD.CEL,GSM239836_AK_SD.CEL,...,ANOVA-adjp,largest fold,430AV2_WB_D2_Enrich,430AV2_WB_D2_poolStDev,430AV2_WB_D2_Cohens_d,430AV2_WB_D2_J,430AV2_WB_D2_Hedges_g,430AV2_WB_D2_Var_d,430AV2_WB_D2_Var_g,430AV2_WB_D2_SEg
MGI symbol,,,,,,,,,,,,,,,,,,,,,


### Columns from the list above can then easily be picked to produce files for use later. Examples below given:
 #### df3 = average S and SD expression for the platform and the log-fold changes
 #### df4 = Hedges g  values and associated variance for Meta-analysis (after indexing)

In [25]:
# df3 = df_FINAL1.loc[:,[u'avg-SD', u'avg-S', u'log_fold-S_vs_SD']]
# df3.columns =[prefix+'avg-SD', prefix+'avg-S', prefix+'log_fold-S_vs_SD']
# df3.to_csv('input_files/430AV2_SymbolExpression_forIndex.csv')

In [26]:
df4 = df_FINAL1.loc[:,[u'430AV2_WB_D2_Enrich',u'430AV2_WB_D2_Hedges_g', u'430AV2_WB_D2_Var_g', u'430AV2_WB_D2_SEg']]
df4.to_csv('../FHS project/Sleep notebook Copy/IPython_notebooks/input_files/430AV2_WB_D2_SymbolforIndexHedges.csv')

In [27]:
df4.head(10)  # check final ouput

,430AV2_WB_D2_Enrich,430AV2_WB_D2_Hedges_g,430AV2_WB_D2_Var_g,430AV2_WB_D2_SEg
MGI symbol,,,,
0610005C13Rik,-0.018220,-0.163728,0.483901,0.695630
0610008F07Rik,-0.141627,-0.603248,0.511992,0.715536
0610009B22Rik,-0.014010,-0.075992,0.482148,0.694369
0610009D07Rik,-0.065213,-0.574990,0.532151,0.728671
0610009O20Rik,-0.200083,-1.482474,0.664811,0.815359
0610010K14Rik,-0.690637,-1.375357,0.639301,0.799563
0610012G03Rik,0.110699,1.917550,1.152717,1.000945
0610031J06Rik,-0.004863,-0.047666,0.481856,0.694158
0610037L13Rik,-0.091457,-0.471744,0.500212,0.707257
